This notebook is intended to demonstrate how vessel segmentation methods of ITKTubeTK can be applied to multi-channel MRA.

In [1]:
import itk
from itk import TubeTK as ttk

from itkwidgets import view

import numpy as np

In [2]:
ImageType = itk.Image[itk.F, 3]

im1 = itk.imread("../Data/MRI-Cases/mra.mha", itk.F)

resampler = ttk.ResampleImage.New( Input=im1, MakeHighResIso=True )
resampler.Update()
im1iso = resampler.GetOutput()


In [3]:
#view(im1iso)

In [9]:
imMath = ttk.ImageMath[ImageType,ImageType].New( Input=im1iso )
imMath.Blur(1)
imBlur = imMath.GetOutput()

numSeeds = 40

In [10]:
vSeg = ttk.SegmentTubes[ImageType].New()
vSeg.SetInput(im1iso)
vSeg.SetVerbose(True)
vSeg.SetMinCurvature(0.0)
vSeg.SetMinRoundness(0.01)
vSeg.SetRadiusInObjectSpace( 1 )
vSeg.SetRadiusMin(1)
vSeg.SetRadiusMax(8)
vSeg.SetDataMinMaxLimits(50,256)
vSeg.SetSeedMask( imBlur )
vSeg.SetUseSeedMaskAsProbabilities(True)
vSeg.SetSeedMaskMaximumNumberOfPoints( numSeeds )

vSeg.ProcessSeeds()

In [12]:
tubeMaskImage = vSeg.GetTubeMaskImage()
#view(tubeMaskImage)
view(image=im1iso, label_image=tubeMaskImage)

Viewer(geometries=[], gradient_opacity=0.22, interpolation=False, point_sets=[], rendered_image=<itk.itkImageP…

In [7]:
SOWriter = itk.SpatialObjectWriter[3].New(vSeg.GetTubeGroup())
SOWriter.SetFileName( "Demo-SegmentBrightVessels.tre" )
SOWriter.Update()